This How I tackled the problem \
Day1: Reading the task and Having no idea how to solve it \
Day2: Searching for techniques for error correction, viewing lecture about Code repitition \
Day3: Trying to undersand how the error will change the repeated qubit code \
Day4: Code Cell1 --> Circuit implementation for a 1 qubit bit_flip error \
Day5-8: Doing other tasks \
Day9: Code Cell2 --> Circuit implementation for a 1 qubit phase_flip error \
Day10-12: Doing other tasks \
Day13: Combining Cell 1 and 2 to make one circuit with possible occurence of both error types \

In [1]:
from qiskit import *
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error
from qiskit import QuantumCircuit, execute, Aer
from qiskit import QuantumRegister, ClassicalRegister

#Ready-Made Noise Model from Qiskit Textbook
def get_noise(p_meas,p_gate):

    error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    error_gate1 = depolarizing_error(p_gate, 1)
    error_gate2 = error_gate1.tensor(error_gate1)

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_meas, "measure") # measurement error is applied to measurements
    noise_model.add_all_qubit_quantum_error(error_gate1, ["x"]) # single qubit gate error is applied to x gates
    noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"]) # two qubit gate error is applied to cx gates
        
    return noise_model

q = QuantumRegister(5,'q')
sb = ClassicalRegister(2,'syndrome_bit')
cb = ClassicalRegister(3,'code_bit')
qc = QuantumCircuit(q,sb,cb)
noise_model = get_noise(0.01,0.01)

# Prepares qubit in the desired initial state
qc.h(q[0])

# Encodes the qubit in a three-qubit entangled state
qc.cx(q[0],q[1])
qc.cx(q[0],q[2])

# Bit-flip error on the second qubit
qc.x(q[1])
qc.x(q[2])

# Adds additional two qubits for error-correction

qc.cx(q[0],q[3])
qc.cx(q[1],q[3])
qc.cx(q[0],q[4])
qc.cx(q[2],q[4])
# Measure the two additional qubits
qc.measure(q[3],sb[0])
qc.measure(q[4],sb[1])


# Do error correction
qc.x(q[1]).c_if(sb, 1)
qc.x(q[2]).c_if(sb, 1)

# Decodes the qubit from the three-qubit entangled state  
qc.cx(q[0], q[1])
qc.cx(q[0], q[2])

# Check the state of the initial qubit
qc.measure(q[0], cb[0])

qc.draw()
counts = execute(qc, Aer.get_backend('qasm_simulator'), noise_model=noise_model).result().get_counts()
print('Results:',counts)


Results: {'000 00': 1, '000 01': 13, '000 10': 15, '000 11': 475, '001 00': 3, '001 01': 17, '001 10': 17, '001 11': 483}


In [2]:
from qiskit import *
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error
from qiskit import QuantumCircuit, execute, Aer
from qiskit import QuantumRegister, ClassicalRegister

#Ready-Made Noise Model from Qiskit Textbook
def get_noise(p_meas,p_gate):

    error_meas = pauli_error([('Z',p_meas), ('I', 1 - p_meas)])
    error_gate1 = depolarizing_error(p_gate, 1)
    error_gate2 = error_gate1.tensor(error_gate1)

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_meas, "measure") # measurement error is applied to measurements
    noise_model.add_all_qubit_quantum_error(error_gate1, ["z"]) # single qubit gate error is applied to Z gates
    noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"]) # two qubit gate error is applied to cx gates
        
    return noise_model

# Creating registers
q = QuantumRegister(5,'q')
sb = ClassicalRegister(2,'syndrome_bit')
cb = ClassicalRegister(3,'code_bit')
# Creates the quantum circuit
qc = QuantumCircuit(q,sb,cb)
#Define Noise Model
noise_model = get_noise(0.01,0.01)


# Prepares qubit in the desired initial state 
qc.h(q[0])

# Encodes the qubit in a three-qubit entangled state  
qc.cx(q[0], q[1])
qc.cx(q[0], q[2])

# Go to Hadamard basis
qc.h(q[0])
qc.h(q[1])
qc.h(q[2])

# Phase error on the second qubit
qc.z(q[1])

# Converts phase error in bit-flip error
qc.h(q[0])
qc.h(q[1])
qc.h(q[2])

# Adds additional two qubits for error-correction
qc.cx(q[0], q[3])
qc.cx(q[1], q[3])
qc.cx(q[0], q[4])
qc.cx(q[2], q[4])

# Measure the two additional qubits
qc.measure(q[3], sb[0])
qc.measure(q[4], sb[1])

# Do error correction
qc.x(q[1]).c_if(sb, 1)
qc.x(q[2]).c_if(sb, 1)

# Decodes the qubit from the three-qubit entangled state  
qc.cx(q[0], q[1])
qc.cx(q[0], q[2])

# Check the state of the initial qubit
qc.measure(q[0], cb[0])



qc.draw()
counts = execute(qc, Aer.get_backend('qasm_simulator'), noise_model=noise_model).result().get_counts()


print('\nThe measured outcomes of the circuits are:', counts)


The measured outcomes of the circuits are: {'000 00': 20, '000 01': 459, '000 10': 2, '000 11': 7, '001 00': 12, '001 01': 518, '001 10': 1, '001 11': 5}
